In [ ]:
#pip install langchain langchain_openai gradio chromadb pypdf langchain-community
#langchain es un framework para trabajar con modelos de lenguaje
#langchain_openai es una extensión de langchain para trabajar con la API de OpenAI
#gradio es una librería para crear interfaces de usuario que nos permite crear testing rapido
#chromadb es una librería que nos permite crear bases de datos vectoriales bajo SQLite
#pypdf es una librería para trabajar con archivos PDF

### LIBRERIAS NECESARIAS

In [61]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma 
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import gradio as gr
#import openai
#import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore") # Ignorar todos los warnings

#### DOCUMENTACION

In [66]:
pdf1 = r"https://www.boe.es/boe/dias/2022/09/29/pdfs/BOE-A-2022-15818.pdf"
pdf2 = r"https://www.camara.es/sites/default/files/publicaciones/fiscalidad03.pdf"
pdf3 = r"https://mpt.gob.es/dam/es/portal/delegaciones_gobierno/delegaciones/madrid/proyectos-ci/Guia-de-Subvenciones-y-Ayudas/231103_Guia_de_Subvenciones_Ayudas_PRTR.pdf.pdf"
pdf4 = r"https://boe.es/buscar/pdf/2014/BOE-A-2014-12328-consolidado.pdf"
pdf5 = r"https://www.aepd.es/guias/gestion-riesgo-y-evaluacion-impacto-en-tratamientos-datos-personales.pdf"
pdf6 = r"https://ipyme.org/PUBLICACIONES_EMPRESAS/Ciclo%20Vital%20de%20la%20Empresa/CreacionEmpresas.pdf"

In [67]:
urls = [pdf1, pdf2, pdf3, pdf4, pdf5, pdf6] #lista de urls de los pdfs

#### PREPROCESAMIENTO DE LA DOCUMENTACION

In [68]:
pages = []
for url in urls:
    loader = PyPDFLoader(url)
    pages += loader.load_and_split()

In [69]:
len(pages) #numero de paginas

931

In [70]:
pages[54] #pagina 54

Document(metadata={'source': 'https://www.boe.es/boe/dias/2022/09/29/pdfs/BOE-A-2022-15818.pdf', 'page': 44}, page_content='b)\u2003Revocación de la autorización.\nc)\u2003Prohibición de solicitar la autorización para operar como plataforma de \nfinanciación participativa por un plazo no inferior a un año ni superior a cinco.\nd)\u2003Prohibición que impida a cualquier miembro del órgano de dirección de la \npersona jurídica responsable de la infracción o a cualquier otra persona física \nconsiderada responsable de la infracción ejercer funciones directivas en \nproveedores de servicios de financiación participativa por plazo no superior a diez \naños.\ne)\u2003Junto con las sanciones previstas en las letras a), b), c) y d), se podrán \nadoptar las siguientes sanciones y medidas administrativas:\n1.º\u2003 Declaración pública en la que consten la persona física o jurídica \nresponsable y la naturaleza de la infracción.\n2.º\u2003 Requerimiento dirigido a la persona física o jurídica pa

#### CONFIGURACION DE LLM

In [71]:
#openai.api_key = os.getenv("OPENAI_API_KEY"),
llm = ChatOpenAI(
    model='gpt-4o',
    api_key=OPENAI_API_KEY,
    temperature=0
)

In [72]:
embedding = OpenAIEmbeddings() #nos permite transformar texto en vectores

#### CREAR UN ALMACEN PARA ESOS VECTORES (DB)

In [73]:
vectorstore = Chroma.from_documents(
    documents = pages,
    embedding = embedding
)

ValueError: Batch size 931 exceeds maximum batch size 166

In [74]:
batch_size = 166  # El tamaño máximo permitido

# Dividir los documentos en lotes más pequeños
for i in range(0, len(pages), batch_size):
    batch = pages[i:i + batch_size]
    vectorstore = Chroma.from_documents(
        documents=batch,
        embedding=embedding
    )

In [75]:
retriever = vectorstore.as_retriever() #nos permite recuperar documentos similares a un texto de entrada

#### PROMTING DE COMPORTAMIENTO

In [76]:
template = """ 

        ###INSTRUCCIONES: 
        Eres un asistente IA del gobierno de España dedicado a responder preguntas empresariales de manera educada y profesional. Debes proporcionar una respuesta útil al usuario. 
        
        En tu respuesta, POR FAVOR SIEMPRE:
          (0) Sé un lector atento a los detalles: lee la pregunta y el contexto y entiende ambos antes de responder.
          (1) Comienza tu respuesta con un tono amigable y reitera la pregunta para que el usuario esté seguro de que la entendiste.
          (2) Si el contexto te permite responder a la pregunta, escribe una respuesta detallada y técnica, útil y fácil de entender, con fuentes referenciadas en el texto. SI NO: no puedes encontrar la respuesta, responde con una explicación, comenzando con: "No pude encontrar la información en las leyes y documentos a los que tengo acceso".
          (3) Debajo de la respuesta, por favor enumera todas las fuentes referenciadas (es decir, párrafos legales que respaldan tus afirmaciones).
          (4) Ahora que tienes tu respuesta, fantástico - revisa tu respuesta para asegurarte de que respondes a la pregunta, esta respuesta es útil y profesional , formateada para ser fácilmente legible.
        
        
        PIENSA PASO POR PASO 
        ###
        
      Responde a la siguiente pregunta utilizando el contexto proporcionado.
        ### Question: {question} ###
        ### Context: {context} ###     
        ### Respuesta Útil con Fuentes:

"""

In [77]:
prompt = PromptTemplate.from_template(template)

#### CADENA O CHAIN

In [78]:
chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

#### PREGUNTA DE CONTROL

In [79]:
ans = chain.invoke("Que documentación necesito para montar una SL en España?")
print(ans)

¡Hola! Entiendo que estás interesado en conocer la documentación necesaria para montar una Sociedad Limitada (SL) en España. A continuación, te detallo los pasos y documentos requeridos para este proceso:

1. **Certificación Negativa del Nombre**: Antes de constituir la sociedad, debes obtener una certificación negativa del nombre en el Registro Mercantil Central. Este documento acredita que no existe otra sociedad con el mismo nombre que deseas utilizar.

2. **Documento de Identidad de los Socios**: Necesitarás el DNI o NIE de todos los socios que formarán parte de la sociedad.

3. **Estatutos Sociales**: Debes redactar los estatutos sociales, que son las normas que regirán el funcionamiento de la sociedad. Estos deben incluir el objeto social, la duración, el domicilio social, y la identidad de los administradores, entre otros aspectos.

4. **Acreditación del Desembolso del Capital Social**: Es necesario demostrar que se ha realizado el desembolso del capital social mínimo requerido,

#### FUNCION DE RESPUESTA

In [80]:
def get_answer(question):
    return chain.invoke(question)

#### INTERFAZ DE PRUEBA

In [81]:
iface = gr.Interface(fn=get_answer, inputs=gr.Textbox(
    value="Pregúntame lo que necesites"),
    live=False, 
    outputs="markdown",  
    title="Inteligencia sobre documentación mediante LLM, APIS y DBVectoriales",
    description="Puedes preguntar cualquier pregunta la creación de empresas en España",
    theme=gr.themes.Soft(),
    allow_flagging="never",)

iface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


#### GUARDAMOS NUESTRA DB CHROMA EN UN ARCHIVO

In [35]:
persist_directory = "chromadb"
vectordb = Chroma.from_documents(documents=pages, embedding=embedding,
                                 persist_directory=persist_directory)
vectordb.persist()